**Rationale** Hypothesis testing is foundational to the entire field of statistics. Without the Central Limit Theorem and hypothesis testing, we would not have modern day (social) science. In marketing, this is generally most important when doing A/B testing of ads. In this assignment, you will practice computing basic summary statistics and conducting hypothesis tests.

[Datasets](https://drive.google.com/drive/folders/1kf6BHlmF32UjsHw6wh_sMJ-JWkV091XD?usp=sharing) required:
1. [FB ad campaign data](https://drive.google.com/file/d/1XqPgu_rFZZ5eHCoFS7drFsmqd7U81vj_/view?usp=sharing).
1. [Starbucks Promos](https://drive.google.com/file/d/1aUJUdGm-RdHaJCJvS1kay9fRdNTq22xb/view?usp=sharing)

# 1. (15 points) FB Ad campaigns

1. Use a groupby operation create a dataframe called `sumstats` consisting of the total impressions and clicks for each `xyz_campaign_id`.
1. Using `sumstats`, create a new column called `ctr` that represents the [click through rate](https://support.google.com/google-ads/answer/2615875?hl=en) for each campaign. Which campaign had the highest CTR?
1. Create a column for `sd` for standard deviation of the click through rate.

Hint: Recall from the notes that the standard deviation of a binomial distribution with $Pr(Success) = p$ is $SD(p) = \sqrt{p(1-p)}$. So if the probability of a binary outcome (such as clicking on an ad) is observed to be .3, the standard deviation is $\sqrt{.3(.7)}=\sqrt{.21}$.

In [ ]:
# imports and mount google drive
import os, pandas as pd, numpy as np
from scipy import stats

from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
# set the path to the datasets for A4
fpath = 'drive/My Drive/Analytics/DataSets/A4/'
os.listdir(fpath)

['facebook_ads.csv', 'starbucks.csv']

In [ ]:
# read in the facebook_ads.csv file as the dataframe variable named fb
fb=pd.read_csv(fpath+'facebook_ads.csv', index_col=0)
fb.shape

(1143, 10)

In [ ]:
# Take a look at the first 5 rows
# Observe that each row represents 1 ad from 1 campaign and a target demographic group (age, gender, interest)
fb.head()

,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
ad_id,,,,,,,,,,
708746,916,103916,30-34,M,15,7350,1,1.43,2,1
708749,916,103917,30-34,M,16,17861,2,1.82,2,0
708771,916,103920,30-34,M,20,693,0,0.00,1,0
708815,916,103928,30-34,M,28,4259,1,1.25,1,0
708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [ ]:
# use a groupby to create new dataframe, sumstats that tabulates the total clicks and impressions for each campaign.
# Generic groupby syntax: sumstats = df.groupby('groupbyvariable')[['summary_variable1', 'summary_variable2']].summaryfunction().reset_index()
# use the correct dataframe name, variables (i.e. column names) and summaryfunction
# Hint: Analyze the different campaigns in 'xyz_campaign_id'

sumstats = fb.groupby('xyz_campaign_id')[['Clicks', 'Impressions']].sum().reset_index()

In [ ]:
# create a new column in sumstats called 'ctr' (short for click through rate)
# recall that ctr = clicks / impressions

sumstats['CTR'] = (sumstats['Clicks'] / sumstats['Impressions'])

In [ ]:
# take a look at the whole sumstats dataframe

fb.shape
print(sumstats)

   xyz_campaign_id  Clicks  Impressions       CTR
0              916     113       482925  0.000234
1              936    1984      8128187  0.000244
2             1178   36068    204823716  0.000176


In [ ]:
# create a new column ctr_sd for the std. dev of the ctr (See hint above)
# pay attention to PEMDAS, esp. that you want to take the sqrt after computing p*(1-p)
# np.sqrt(x) returns the sqrt of some number x
p = sumstats['CTR']
sumstats['CTR_SD'] = np.sqrt(p * (1 - p))

In [ ]:
# take another look at the whole sumstats dataframe
# compare the ctr_std to calculating it by hand, make sure you did this correctly.
print(sumstats)


   xyz_campaign_id  Clicks  Impressions       CTR    CTR_SD
0              916     113       482925  0.000234  0.015295
1              936    1984      8128187  0.000244  0.015621
2             1178   36068    204823716  0.000176  0.013269


## Answer (edit this cell)
The campaign with the highest CTR is: **_Campaign  1 (xyz id is 936)_**


# 2. (15 points) Hypothesis testing and confidence intervals

1. Compute the 95% confidence interval for the CTR for each campaign. Compare the confidence intervals of campaign 916 and 936. What can you conclude about the relative perfrmance of the 2 ads in the population (e.g. are they very different, similar, etc.)? How about 916 vs. 1178?
1. Was campaign 936 statistically different compared to campaign 916? How about 936 vs. 1178? Use the `ttest_2sample` function from the notes. Remember, you must define it in your Colab session in order to use it (execute the cell w/ the function).

Given these statistical tests what would you recommend in terms of allocation of the ad budget?

In [ ]:
# use stats.norm.ppf([.025, .975], mean, std error)
# where mean is sample mean and std error is std dev / sqrt(obs) (from Central Limit Theorem)

sumstats['STD_ERR'] = sumstats['CTR_SD']/(np.sqrt(sumstats['Impressions']))

# confidence interval for ad 916

CI0 = stats.norm.ppf([0.025, 0.975], sumstats['CTR'][0],sumstats['STD_ERR'][0])

print(CI0)

[0.00019085 0.00027713]


In [ ]:
CI916 = CI0[1]-CI0[0] #916
print(CI916)

8.627536464879144e-05


In [ ]:
# confidence interval for ad 936

CI1 = stats.norm.ppf([0.025, 0.975], sumstats['CTR'][1],sumstats['STD_ERR'][1])
print(CI1)

[0.00023335 0.00025483]


In [ ]:
CI936 = CI1[1]-CI1[0] #936
print(CI936)

2.147841474612873e-05


In [ ]:
# confidence interval for ad 1178

CI2 = stats.norm.ppf([0.025, 0.975], sumstats['CTR'][2],sumstats['STD_ERR'][2])
print(CI2)

[0.00017428 0.00017791]


In [ ]:
CI1178 = CI2[1]-CI2[0] #1178
print(CI1178)

3.634298833543278e-06


In [ ]:
sumstats.head()

,xyz_campaign_id,Clicks,Impressions,CTR,CTR_SD,STD_ERR
0,916,113,482925,0.000234,0.015295,2.200943e-05
1,936,1984,8128187,0.000244,0.015621,5.479288e-06
2,1178,36068,204823716,0.000176,0.013269,9.271341e-07


In [ ]:
print(CI916,CI936,CI1178)

8.627536464879144e-05 2.147841474612873e-05 3.634298833543278e-06


**Edit this cell**

1. (write 1-2 sentences to compare the confidence interval of 916 vs 936's CTR)

**916 has a larger confidence interval than 936, which means that there is more variation in 916 CTR.**

1. (write 1-2 sentences to compare the confidence interval of 916 vs 1178's CTR)

**1178 has a smaller confidence interval than 916. This means that 916 has the widest range of CTR's**

In [ ]:
# Use this custom funciton

def ttest_2sample(m1,sd1,N1,m2,sd2,N2, twotail = True, equalvar = False):
    """
    This function requires you to supply:
    m1: mean of sample 1
    sd1: std. dev of sample 1
    N1: number of obs of sample 1
    m2: mean of sample 2
    sd2: std dev of sample 2
    N2: number of obs of sample 2

    Optional inputs:
    twotail = True (default) / False. If False, then 1 tail
    equalvar = True / False (default). If True, assumes equal population variance.
    """

    # The difference between equal and unequal variance is only in how to compute
    # the test statistic and degree of freedom.
    if equalvar:
        spsquare = ((N1-1)*sd1**2+(N2-1)*sd2**2)/(N1+N2-2)
        T = (m1-m2)/np.sqrt(spsquare*(1/N1+1/N2))
        nu = N1+N2-2
    else:
        nu = (sd1**2/N1+sd2**2/N2)**2/((sd1**2/N1)**2/(N1-1)+(sd2**2/N2)**2/(N2-1)) # new degree of freedom
        T = (m1-m2)/(np.sqrt(sd1**2/N1+sd2**2/N2))

    # If the first mean is bigger, we need to do 1- cdf
    # Otherwise just compute cdf
    if m1>m2:
        pval = 1-stats.t.cdf(T, df = nu)
    else:
        pval = stats.t.cdf(T, df = nu)

    # return p values
    # If 2 tail, we must multiply by 2
    # otherwise we just return the computed pval
    if twotail == True:
        return pval*2
    else:
        return pval

In [ ]:
# for each campaign grab the Pr(Success) = p which is ctr, std dev, and observations.
# p916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'ctr'])
# sd916 = np.sqrt(p916*(1-p916))
# obs916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'Impressions'])

#916
p916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'CTR'])
sd916 = np.sqrt(p916*(1-p916))
obs916 = float(sumstats.loc[sumstats.xyz_campaign_id==916, 'Impressions'])

#936
p936 = float(sumstats.loc[sumstats.xyz_campaign_id==936, 'CTR'])
sd936 = np.sqrt(p936*(1-p936))
obs936 = float(sumstats.loc[sumstats.xyz_campaign_id==936, 'Impressions'])

#1178
p1178 = float(sumstats.loc[sumstats.xyz_campaign_id==1178, 'CTR'])
sd1178 = np.sqrt(p1178*(1-p1178))
obs1178 = float(sumstats.loc[sumstats.xyz_campaign_id==1178, 'Impressions'])


# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 916

ttest_2sample(p916,sd916,obs916,p936,sd936,obs936, twotail = True, equalvar = False)


0.6561623087350006

In [ ]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 1178

ttest_2sample(p936,sd936,obs936,p1178,sd1178,obs1178, twotail = True, equalvar = False)

0.0

**Edit this cell to answer**

In order to have the widest range of coverage with ads, it would be best to run both 936 and 1178 because there is no statistical relationship between the two, whereas campaign ads 916 and 936 are statistically significant.

# 3. (20 points)

Using the Starbucks dataset, conduct an appropriate statistical test to determine:
1. Is there any statistically appreciable difference in the redemption rates of BOGO vs. Discount promotions?
1. Among those who redeemed an offer and reported their income, is there any statistical difference in the average income associated with BOGO vs. discount redemptions? Note that not every observation reports income (select only the observations that do).

In [ ]:
# read in the starbucks_promos.csv file as the dataframe variable named sb, use index_col = 0 option
starb = pd.read_csv(fpath + 'starbucks.csv')

In [ ]:
# preview first 5 rows to get a sense of the contents of the dataframe
# note that each observation represents an offer received by a customer
starb.head()

,Unnamed: 0,uid,event,time,gender,age,register_date,income,offer_id,offer_reward,channels,difficulty,duration,offer_type,offer_time,transaction_amount,redeem_time,redeemed
0,1,0020c2b971eb4e9188eac86d93036a77,offer received,0,F,59,20160304,90000.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,"['web', 'email', 'mobile', 'social']",10.0,240.0,discount,0.0,17.63,54.0,1
1,4,005500a7188546ff8a767329a2f7c76a,offer received,0,M,56,20171209,47000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,NaN,NaN,0
2,5,0056df74b63b4298809f0b375a304cf4,offer received,0,M,54,20160821,91000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"['web', 'email', 'mobile']",5.0,168.0,bogo,0.0,27.86,132.0,1
3,6,00715b6e55c3431cb56ff7307eb19675,offer received,0,F,58,20171207,119000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,27.26,12.0,1
4,8,00840a2ca5d2408e982d56544dc14ffd,offer received,0,M,26,20141221,61000.0,2906b810c7d4411798c6938adc9daaa5,2.0,"['web', 'email', 'mobile']",10.0,168.0,discount,0.0,6.05,540.0,1


In [ ]:
# select the redeemed column for discount offers types, name this data the variable disc
# ( e.g. disc = sb.loc[sb.offer_type=='discount', 'redeemed'] )

disc = starb.loc[starb.offer_type=='discount', 'redeemed']

In [ ]:
# select the redeemed column for buy one get one (bogo) offers, name this data the variable bogo

bogo = starb.loc[starb.offer_type=='bogo', 'redeemed']

In [ ]:
# use stats.ttest_ind to test whether the 2 types of offers yield different redemption rates

ttest = stats.ttest_ind(bogo,disc)
print(ttest)

TtestResult(statistic=-17.30160421286854, pvalue=6.611390792545181e-67, df=61040.0)


In [ ]:
# Select the income column for those who redeemed the BOGO offer, call this the variable bogo_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for bogo offer, and one for income not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data

bogo_income = starb.loc[(starb.offer_type=='bogo')&(starb.redeemed==1)&(starb.income.notnull()==True), 'income']


In [ ]:
# Select the income column for those who redeemed the discount offer, call this the variable disc_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for discount offer,  and one for income not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data

disc_income = starb.loc[(starb.offer_type=='discount')&(starb.redeemed==1)&(starb.income.notnull()==True), 'income']

In [ ]:
# What is the average income for bogo redeemers (bogo_income.mean())?

(bogo_income.mean())

70114.8559593297

In [ ]:
# What is the average income for discount redeemers?

(disc_income.mean())

68617.39130434782

In [ ]:
# use stats.ttest_ind to test whether the average income of the redeemers of the 2 offer types are different

ttest2 = stats.ttest_ind(bogo_income,disc_income)
ttest2


TtestResult(statistic=6.343377852366808, pvalue=2.2763463492984577e-10, df=33641.0)

**Edit this cell**
1. The low p-value indicates that these campaigns serve different purposes and attract different types of customers.

1. The low p-value shows that statisically there is a difference in level of income between the bogo and discount group.